In [ ]:
import cobra
from cobra.io import read_sbml_model
from cobra import Model, Reaction, Metabolite
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
pa74 = read_sbml_model('../PA/Pediococcus_acidilactici_7_4.xml')


In [ ]:
M9 = {'EX_ca2_e': 1000.0, 'EX_cl_e': 1000.0, 'EX_co2_e': 1000.0, 'EX_cobalt2_e': 1000.0, 
      'EX_cu2_e': 1000.0, 'EX_fe2_e': 1000.0, 'EX_fe3_e': 1000.0, 'EX_h_e': 1000.0, 'EX_h2o_e': 1000.0, 
      'EX_k_e': 1000.0, 'EX_mg2_e': 1000.0, 'EX_mn2_e': 1000.0, 'EX_mobd_e': 1000.0, 'EX_na1_e': 1000.0, 
      'EX_zn2_e': 1000.0, 'EX_ni2_e': 1000.0, 'EX_glc__D_e': 20.0,'EX_so4_e': 1000.0, 
      'EX_nh4_e': 1000.0, 'EX_pi_e': 1000.0, 'EX_o2_e': 20.0}

## PedA synthesis

'H-Lys-Tyr-Tyr-Gly-Asn-Gly-Val-Thr-Cys(1)-Gly-Lys-His-Ser-Cys(1)-Ser-Val-Asp-Trp-Gly-Lys-Ala-Thr-Thr-Cys(2)-Ile-Ile-Asn-Asn-Gly-Ala-Met-Ala-Trp-Ala-Thr-Gly-Gly-His-Gln-Gly-Asn-His-Lys-Cys(2)-OH'

https://pubchem.ncbi.nlm.nih.gov/compound/Pediocin-PA-1#section=Biologic-Description


In [ ]:
# pediocin PA-1
aa_freq = {}
pedA_seq = 'MKKIEKLTEKEMANIIGGKYYGNGVTCGKHSCSVDWGKATTCIINNGAMAWATGGHQGNHKC'
for a in pedA_seq:
    if a in aa_freq:
        aa_freq[a] += 1
    else:
        aa_freq[a] = 1
print(aa_freq)
    
atp_num = len(pedA_seq)*4.306
print(atp_num)

In [ ]:
aa_names = {'A':'ala__L_c', 'C':'cys__L_c', 'D':'asp__L_c', 'E':'glu__L_c', 'F':'phe__L_c', 'G':'gly_c',
            'H':'his__L_c', 'I':'ile__L_c', 'K':'lys__L_c', 'L':'leu__L_c', 'M':'met__L_c', 'N':'asn__L_c',
            'P':'pro__L_c','Q':'gln__L_c', 'R':'arg__L_c', 'S':'ser__L_c', 'T':'thr__L_c', 'V':'val__L_c',
            'W':'trp__L_c', 'Y':'tyr__L_c'}
pedA_c = Metabolite('pedA_c',formula='C196H293N61O60S5',name='Pediocin PA-1',compartment='C_c')
pedA_e = Metabolite('pedA_e',formula='C196H293N61O60S5',name='Pediocin PA-1',compartment='C_e')
pa.add_metabolites([pedA_c,pedA_e])
#pediocin synthesis
PedA_synthesis = Reaction('PedA_synthesis')
PedA_synthesis.name = 'Pediocin PA-1 synthesis'
PedA_synthesis.subsystem = ''
PedA_synthesis.lower_bound = 0
PedA_synthesis.upper_bound = 1000.
PedA_synthesis.gene_reaction_rule = ''
PedA_synthesis_rxn = ''
for aa in aa_freq:
    PedA_synthesis_rxn = PedA_synthesis_rxn + str(float(aa_freq[aa])) + ' ' + aa_names[aa] + ' + '
PedA_synthesis_rxn = PedA_synthesis_rxn + '266.972 atp_c --> 266.972 adp_c + 266.972 pi_c + pedA_c'
print(PedA_synthesis_rxn)
#pediocin export: c->e and exchange
PedAabc = Reaction('PedAabc')
PedAabc.name = 'Pediocin PA-1 export via ABC system'
PedAabc.subsystem = ''
PedAabc.lower_bound = 0
PedAabc.upper_bound = 1000.
PedAabc.gene_reaction_rule = ''
PedAabc_rxn = 'pedA_c + atp_c + h2o_c --> pedA_e + adp_c + pi_c + h_c '
EX_pedA_e = Reaction('EX_pedA_e')
EX_pedA_e.name = 'R_EX_pedA_e'
EX_pedA_e.subsystem = ''
EX_pedA_e.lower_bound = -1000.
EX_pedA_e.upper_bound = 1000.
EX_pedA_e.gene_reaction_rule = ''
EX_pedA_e_rxn = 'pedA_e <=> '
pa.add_reactions([PedA_synthesis,PedAabc,EX_pedA_e])
PedA_synthesis.build_reaction_from_string(PedA_synthesis_rxn)
PedAabc.build_reaction_from_string(PedAabc_rxn)
EX_pedA_e.build_reaction_from_string(EX_pedA_e_rxn)